In [ ]:
from src.scraping.config import HEADERS, URL_HITTER, URL_PITCHER, URL_POST
import requests
from bs4 import BeautifulSoup

In [ ]:
# Get _RequestVerificationToken
session = requests.Session()
res = session.get(URL_HITTER, headers=HEADERS)
if res.status_code == 200:
    soup = BeautifulSoup(res.text, "html.parser")
    print(res.status_code)
else:
    print(res.status_code)
    
token = soup.find("input", {"name": "__RequestVerificationToken"})["value"]

In [ ]:
# 取得th tag，也就是欄位名稱
columns = [th.get_text(strip=True) for th in soup.select("tr th")]
columns[0] = "球員"
columns

In [ ]:
# 爬取從第2頁開始的打者資料
max_pages = 200
year = 2025
hitters = []
for page in range(1, max_pages):
    
    payload = {
        "__RequestVerificationToken": token,
        "ExecAction": "Q",
        "IndexOfPages": str(page),
        "Sortby": "9",
        "KindCode": "A",
        "Year": str(year),
        "Online": "01",
        "Position": "01",
        "DefenceType": "99",
        "PageSize": "15"
    }
    
    res = session.post(URL_POST, headers=HEADERS, data=payload, allow_redirects=False)
    soup = BeautifulSoup(res.text, "html.parser")
    
    # 判斷是否有資料
    tds = soup.select("tr td")
    if not tds:
        print(f"No more data at page {page}, stop.")
        break
    
    # 解析資料
    for td in tds:
        hitters.append(td.get_text(strip=True))

hitters = [hitters[i: i + 33] for i in range(0, len(hitters), 33)]  
hitters